In [ ]:
!pip install optuna

In [ ]:
import shap
import optuna
from sklearn import set_config
from xgboost import XGBRegressor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

# Dataset

In [ ]:
set_config(transform_output="pandas")

In [ ]:
# load the data

data = fetch_california_housing(as_frame=True)

In [ ]:
data.keys()

In [ ]:
# make X and y

X, y = data.data, data.target

In [ ]:
# columns names

feature_names = data.feature_names

feature_names

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X.shape

In [ ]:
# split the data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of Training data: ", X_train.shape)
print("Shape of Testing data: ", X_test.shape)

# Model

In [ ]:
# model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42,
                         n_jobs=-1, learning_rate=0.2, n_estimators=200,
                         reg_lambda=50, max_depth=7, gamma=0.05)

In [ ]:
# fit the training data
xgb_model.fit(X_train, y_train)

In [ ]:
# claculate the predictions
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# calculate the RMSE and R2 score
rmse_train = root_mean_squared_error(y_train, y_pred_train)
rmse_test = root_mean_squared_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train RMSE: {rmse_train:.4f}, R2: {r2_train:.4f}")
print(f"Test RMSE: {rmse_test:.4f}, R2: {r2_test:.4f}")

# HP Tuning

In [ ]:
# tune the model

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 1, 100),
        'gamma': trial.suggest_float('gamma', 0, 2),
        "subsample": trial.suggest_float('subsample', 0.7, 1.0),
        "n_jobs": -1,
        'objective': 'reg:squarederror',
        'random_state': 42
    }

    # set the model with the parameters
    model = XGBRegressor()

    model.set_params(**params)

    # fit the model
    model.fit(X_train, y_train)

    # predict the test data
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

In [ ]:
# create a study
study = optuna.create_study(direction='maximize')

study.optimize(func=objective, n_trials=50, show_progress_bar=True, n_jobs=-1)

In [ ]:
# get the best parameters
study.best_params

In [ ]:
# get the best value
study.best_value

In [ ]:
# get the best estimator
best_model = XGBRegressor(**study.best_params)

best_model.fit(X_train, y_train)

In [ ]:
# print the metrics
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

rmse_train = root_mean_squared_error(y_train, y_pred_train)
rmse_test = root_mean_squared_error(y_test, y_pred_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train RMSE: {rmse_train:.4f}, R2: {r2_train:.4f}")
print(f"Test RMSE: {rmse_test:.4f}, R2: {r2_test:.4f}")

# SHAP

In [ ]:
best_model

In [ ]:
# create the tree explainer

explainer = shap.TreeExplainer(model=best_model,
                               data=X_train)

In [ ]:
# expected value
explainer.expected_value

In [ ]:
# calculate the shap values for all test data

shap_values = explainer(X_test)

In [ ]:
shap_values.shape

In [ ]:
len(feature_names)

In [ ]:
feature_names

In [ ]:
shap_values[0:5]

In [ ]:
X_test.head(1).values

# GLOBAL PLOTS

In [ ]:
shap_values

In [ ]:
shap_values.shape

In [ ]:
# bar plot to plot feature importance

shap.plots.bar(shap_values)

In [ ]:
shap_values.abs.mean(axis=0)

In [ ]:
shap_values.abs.mean(axis=0).values

In [ ]:
dict(zip(feature_names,shap_values.abs.mean(axis=0).values))

In [ ]:
feature_importances = {feature:shap_val.item() for feature, shap_val in
                    zip(feature_names,shap_values.abs.mean(axis=0).values)}

In [ ]:
feature_importances

In [ ]:
sorted(feature_importances, key=feature_importances.get, reverse=True)

In [ ]:
# heatmap plot

shap.plots.heatmap(shap_values)

In [ ]:
# summary plot

shap.plots.beeswarm(shap_values)

In [ ]:
# scatter plot (Dependence Plot)
shap.plots.scatter(shap_values[:, "MedInc"])

In [ ]:
shap_values[:, "Latitude"]

In [ ]:
shap.plots.scatter(shap_values[:, "AveOccup"])

In [ ]:
shap.plots.scatter(shap_values[:, "HouseAge"])

In [ ]:
# scatter plot with interaction

shap.plots.scatter(shap_values[:, "HouseAge"], color=shap_values)

# LOCAL PLOTS

In [ ]:
test_case = X_test.sample(1)

test_case

In [ ]:
explained_row = explainer(test_case)

explained_row

In [ ]:
explained_row[0,:].shape

In [ ]:
explained_row.shape

In [ ]:
# waterfall plot

shap.plots.waterfall(explained_row[0])

In [ ]:
# force plot

shap.plots.initjs()

shap.plots.force(explained_row[0])

In [ ]:
# bar plot for local explaination

shap.plots.bar(explained_row[0])

# LIME

In [ ]:
# test case

test_case

In [ ]:
# convert test case to a 1D array

test_case_lime = test_case.values.flatten()

In [ ]:
!pip install lime

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
X_train.values.shape

In [ ]:
# feature names

feature_names = X_train.columns.tolist()

In [ ]:
# make an lime explainer

lime_explainer = LimeTabularExplainer(
    training_data=X_train.values,
    mode="regression",
    feature_names=feature_names
)

In [ ]:
# get the lime explanations

lime_explanation = lime_explainer.explain_instance(
    data_row=test_case_lime,
    predict_fn=xgb_model.predict
)

In [ ]:
# lime explanations --> list of values

lime_explanation.as_list()

In [ ]:
# lime explanations --> pyplot figure

lime_explanation.as_pyplot_figure()

In [ ]:
# lime explanations --> as html

from IPython.core.display import HTML

In [ ]:
xgb_model.predict(test_case)

In [ ]:
# display explanation

display(HTML(lime_explanation.as_html()))
